# Lasso Scenario Creation Quickstart

In this notebook we will run through:

1. Using a configuration file to run lasso  
2. Setting up a base scenario and applying projects  
3. Transforming the standard network format to the MetCouncil expected format    
4. Exporting the network to a shapefile and csvs  

In [ ]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

# Configuring  a Scenario

There are three places where scenario parameters live, each one overriding the previous:
### 1. [**Default parameters** as specified in `Parameters.py`](https://wsp-sag.github.io/Lasso/_generated/lasso.Parameters.html#lasso.Parameters).  

The defaults are specified here as reasonable assumptions for MetCouncil, but should be examined.

```python
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    my_scenario.road_net,
)
```


### 2. **Config File** 
A config file is a yaml-formatted file which provides a separate place (from your code) to document configurations and parameters. Methods which have a parameters option can be fed the config file (i.e. `parameters=my_config.get("my_parameters", {})`) to have all the default parameters updated for the purposes of that method rather than having to specify each of them separately.  

**Using the config file to override default parameters in bulk**

```python
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    my_scenario.road_net, parameters=my_config.get("my_parameters", {})
)
```

**Using the config file to store information outside of code**
The config file can also store other file location information (i.e. input network) that you don't want to have in your code itself. 

```python
base_scenario = Scenario.create_base_scenario(
        my_config["base_scenario"]["shape_file_name"],
        my_config["base_scenario"]["link_file_name"],
        my_config["base_scenario"]["node_file_name"],
        roadway_dir=os.path.join(base_wrangler_path,my_config["base_scenario"]["input_dir"]),
        transit_dir=os.path.join(base_wrangler_path,my_config["base_scenario"]["input_dir"]),
        validate=False
    )
```

###  3. **Method or Function Keyword Arguments** 
This method will override any default parameters or those fed in with the bulk `parameters =` keyword argument for individual methods. 

```python
model_road_net.write_roadway_as_shp(
    output_link_shp = "my_own_link_shp.shp",
    output_node_shp = "my_own_link_shp.shp",
)
```

## Config File

Let's examine the configuration file and store it as `my_config` variable.  

Configuration files are written in YAML and read in as python dictionaries.

In [ ]:
MY_CONFIG_FILE = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "settings","my_config.yaml"
)

with open(MY_CONFIG_FILE) as f:
        my_config = yaml.safe_load(f)
        
import json
print(json.dumps(my_config, indent=2))

## Alternatively this could be written as a dictionary created in the notebook our selected via a notebook GUI

## Create a Base Scenario

Base scenarios must at the least specify a highway network but can also specify a directory where transit networks can be found.  

In this step the highway and transit networks are read in and validated to each other – making sure that transit nodes exist and the routes can be followed along the roadway network..  

Validation can take some time, so if you are already confident that your roadway and transit networks are valid and compatible you may want to override the validation using the flag: `validate = False`.

In [ ]:
base_wrangler_path = os.path.join(os.path.dirname((os.path.dirname(os.path.abspath('')))),"network_wrangler")
WranglerLogger.info("Base Wrangler Path: {}".format(base_wrangler_path))

base_scenario = Scenario.create_base_scenario(
        my_config["base_scenario"]["shape_file_name"],
        my_config["base_scenario"]["link_file_name"],
        my_config["base_scenario"]["node_file_name"],
        roadway_dir=os.path.join(base_wrangler_path,my_config["base_scenario"]["input_dir"]),
        transit_dir=os.path.join(base_wrangler_path,my_config["base_scenario"]["input_dir"]),
        validate=False
    )

#### Create project cards from projects that are explicitely specified in config


In [ ]:
if len(my_config["scenario"]["project_cards_filenames"]) > 0:
    project_cards_list = [
        ProjectCard.read(filename, validate=False)
        for filename in my_config["scenario"]["project_cards_filenames"]
    ]
else: 
    project_cards_list = []
project_cards_list

## Create Scenario

A scenario is constructed with a base scenario and then selecting project cards to be added to that base scenario to create the new scenario using the command `my_scenario = Scenario.create_scenario()`

Projects can be added using the following combination of keyword arguments within `Scenario.create_scenario()`:

 1. `card_directory` + `tags` will search a directory add project's who's project tags match *at least one of* the tags in the keyword.
 2. `card_directory` + `glob_search` will search a directory add project's who's file name matches the [glob search text](https://docs.python.org/3/library/glob.html)
 3. `project_cards_list` is a list of ProjectCard objects
 
Be default, project cards added in `Scenario.create_scenario()` are validated to the project card schema when they are  read in.  However, since validation can take some time you may wish to override the validation using the flag `validate_project_cards` if you are already confident that your project cards are valid.

   `validate_project_cards = False`
   
Projects that are not added to the scenario instance (i.e. `my_scenario`) when it is created from `Scenario.create_scenario()` can be added to the scenario instane later by using the following helper methods:  

 - `my_scenario.add_project_card_from_file()`  
 - `my_scenario.add_project_cards_from_directory()`  
 - `my_scenario.add_project_cards_from_tags()`
 
...or by directly adding a ProjectCard intance (i.e. `my_project`) to the scenario instance's `project` attribute:

```python
my_project = ProjectCard.read(path_to_project_card)
my_scenario.projects += my_project

```


In [ ]:
my_scenario=None

my_scenario = Scenario.create_scenario(
    base_scenario=base_scenario,
    card_directory=os.path.join(base_wrangler_path,my_config["scenario"]["card_directory"]),
    tags=my_config["scenario"]["tags"],
    project_cards_list=project_cards_list,
    glob_search=my_config["scenario"]["glob_search"],
    validate_project_cards=False,
)

### Apply all projects in scenario

In [ ]:
WranglerLogger.info("\nProjects in queue to be applied: \n - {}".format("\n - ".join(my_scenario.get_project_names())))
WranglerLogger.info("\n[Before] Applied Projects: \n - {}".format("\n - ".join(my_scenario.applied_projects)))

my_scenario.apply_all_projects()

WranglerLogger.info("\n[After] Applied Projects: \n - {}".format("\n - ".join(my_scenario.applied_projects)))

# Write out as MetCouncil Model Roadway Network
Everything above was done in "pure wrangler" rather than lasso.  However, we will need Lasso in order to add the MetCouncil specific variables. You can create a lasso ModelRoadwayNetwork object from the roadway network object and feed it any additional parameters from that `my_config` variable.

You can see that the link variables for this network are the same as the standard roadway network at this point but that will change.

Since this is a GeoDataFrame you can also use build-in Geopandas features to make simple plots based on these variables.

In [ ]:
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    my_scenario.road_net, parameters=my_config.get("my_parameters", {})
)

WranglerLogger.info("\nmodel_road_net columns:\n - {}".format("\n - ".join(model_road_net.links_df.columns)))

In [ ]:
model_road_net.links_df.geom_type

In [ ]:
model_road_net.links_df.plot("bike_access")

## Add MetCouncil variables
At this point, we need to calculate all the variables into what MetCouncil's model is expecting. The method `roadway_standard_to_met_council_network()` broadly does the following:  
 
- creates a parallel managed lane network
- calculates additional variables based on geography or other variables (i.e. county, assignment group, area type, etc)
- flattens variables stored as continuous time values and determines their value by time period (i.e. lanes_am)   
- reprojects into MetCouncil's projection

In [ ]:
model_road_net.roadway_standard_to_met_council_network()

In [ ]:
WranglerLogger.info("\nmodel_road_net **links_metcouncil** columns:\n - {}".format("\n - ".join(model_road_net.links_metcouncil_df.columns)))

In [ ]:
model_road_net.links_metcouncil_df.plot("lanes_AM")

## Export to shapefile

As a last step, the network can be exported to a shapefile and paired CSVs after removing extraneous variables.

(note that this step will also run the `roadway_standard_to_met_council_network()` method but I wanted to show it to you piecewise)

In [ ]:
model_road_net.write_roadway_as_shp()

# Export to fixed width file

In [ ]:
model_road_net.write_roadway_as_fixedwidth()

# Write out as MetCouncil Model Transit Network

Similar to the roadway network, the first step is to convert it to a Lasso object, and then write it to a cube line file.  Optionally, you could also export it to a shapefile to inspect using other means. 

In [ ]:
standard_transit = StandardTransit.fromTransitNetwork(my_scenario.transit_net)
standard_transit.feed

Write out the StandardTransit Lasso object to a cube line file:

In [ ]:
standard_transit.write_as_cube_lin()

In [ ]:
BASE_ROADWAY_DIRECTORY = ''
BASE_TRANSIT_DIRECTORY = ''